# Clustering Crypto

In [139]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [140]:
# Load the crypto_data.csv dataset.
file_path='crypto_data.csv'
crypto_df=pd.read_csv(file_path,index_col="Unnamed: 0")
crypto_df

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...
XBC,BitcoinPlus,Scrypt,True,PoS,1.283270e+05,1000000
DVTC,DivotyCoin,Scrypt,False,PoW/PoS,2.149121e+07,100000000
GIOT,Giotto Coin,Scrypt,False,PoW/PoS,NaN,233100000
OPSC,OpenSourceCoin,SHA-256,False,PoW/PoS,NaN,21000000


In [141]:
crypto_df.dtypes


CoinName            object
Algorithm           object
IsTrading             bool
ProofType           object
TotalCoinsMined    float64
TotalCoinSupply     object
dtype: object

In [142]:
# Keep all the cryptocurrencies that are being traded.
crypto_df = crypto_df.loc[crypto_df["IsTrading"]==True]
crypto_df

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...
SERO,Super Zero,Ethash,True,PoW,NaN,1000000000
UOS,UOS,SHA-256,True,DPoI,NaN,1000000000
BDX,Beldex,CryptoNight,True,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,True,PoW,7.296538e+06,21000000


In [143]:
# Keep all the cryptocurrencies that have a working algorithm.
#Use isnull() to check for null values
crypto_df.isnull().sum()
#No null values found, therefore all have a working algorithm, therefore none need to be dropped

CoinName             0
Algorithm            0
IsTrading            0
ProofType            0
TotalCoinsMined    459
TotalCoinSupply      0
dtype: int64

In [144]:
# Remove the "IsTrading" column. 
crypto_df = crypto_df.drop(["IsTrading"],axis=1)
crypto_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...
SERO,Super Zero,Ethash,PoW,NaN,1000000000
UOS,UOS,SHA-256,DPoI,NaN,1000000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [145]:
# Remove rows that have at least 1 null value.
crypto_df = crypto_df.dropna()
crypto_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
...,...,...,...,...,...
ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [146]:
# Keep the rows where coins are mined.
crypto_df = crypto_df.loc[crypto_df["TotalCoinsMined"]>0]
crypto_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
...,...,...,...,...,...
ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [147]:
# Create a new DataFrame that holds only the cryptocurrencies names.
names_df = crypto_df[["CoinName"]]
names_df

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum
...,...
ZEPH,ZEPHYR
GAP,Gapcoin
BDX,Beldex
ZEN,Horizen


In [148]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_df = crypto_df.drop(["CoinName"],axis=1)
crypto_df

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0
...,...,...,...,...
ZEPH,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Equihash,PoW,7.296538e+06,21000000


In [149]:
# Use get_dummies() to create variables for text features.
crypto_binary = pd.get_dummies(crypto_df, columns=["Algorithm","ProofType"])
crypto_binary

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZEPH,2.000000e+09,2000000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GAP,1.493105e+07,250000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BDX,9.802226e+08,1400222610,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ZEN,7.296538e+06,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [150]:
#grab index values to reinsert after pca
crypto_id = []
row_num = len(crypto_binary)
row_num
for i in range(row_num):
    crypto_id.append(crypto_binary.index[i])
    
crypto_id

['42',
 '404',
 '1337',
 'BTC',
 'ETH',
 'LTC',
 'DASH',
 'XMR',
 'ETC',
 'ZEC',
 'BTS',
 'DGB',
 'BTCD',
 'XPY',
 'PRC',
 'KOBO',
 'SPR',
 'ARG',
 'AUR',
 'BLU',
 'XMY',
 'MOON',
 'ZET',
 'SXC',
 'QTL',
 'ENRG',
 'QRK',
 'RIC',
 'DGC',
 'BTB',
 'CAT',
 'CBX',
 'CCN',
 'CRYPT',
 'CSC',
 'DMD',
 'XVG',
 'DVC',
 'EAC',
 'EFL',
 'EMC2',
 'EMD',
 'EXCL',
 'FLT',
 'FRK',
 'FTC',
 'GDC',
 'GLC',
 'GLD',
 'HBN',
 'HYP',
 'IFC',
 'IOC',
 'IXC',
 'KGC',
 'LKY',
 'LTB',
 'MAX',
 'MEC',
 'MED',
 'MINT',
 'MINC',
 'MZC',
 'NAUT',
 'NAV',
 'NOBL',
 'NMC',
 'NYAN',
 'OPAL',
 'ORB',
 'POT',
 'PXC',
 'RDD',
 'RPC',
 'SBC',
 'SMC',
 'SUPER',
 'SYNC',
 'SYS',
 'TES',
 'TGC',
 'TIT',
 'TOR',
 'TRC',
 'UNB',
 'UNO',
 'URO',
 'USDE',
 'UTC',
 'VIA',
 'VRC',
 'VTC',
 'WDC',
 'XC',
 'XCR',
 'XJO',
 'XST',
 'ZCC',
 'BCN',
 'XDN',
 'BURST',
 'SJCX',
 'MONA',
 'NTRN',
 'FAIR',
 'NLG',
 'RBY',
 'PTC',
 'KORE',
 'WBB',
 'NOTE',
 'FLO',
 '8BIT',
 'STV',
 'ABY',
 'FLDC',
 'U',
 'UIS',
 'CYP',
 'OMC',
 'VTR',
 'GRE'

In [151]:
# Standardize the data with StandardScaler().
crypto_scaled = StandardScaler().fit_transform(crypto_binary)
crypto_scaled

array([[-0.11710817, -0.1528703 , -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.09396955, -0.145009  , -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [ 0.52494561,  4.48942416, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       ...,
       [-0.09561336, -0.13217937, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.11694817, -0.15255998, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.11710536, -0.15285552, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ]])

### Deliverable 2: Reducing Data Dimensions Using PCA

In [152]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components = 3,random_state = 0)
crypto_pca = pca.fit_transform(crypto_scaled)
crypto_pca

array([[-0.33362577,  1.0175199 , -0.60146434],
       [-0.31696777,  1.01768948, -0.60179865],
       [ 2.29228702,  1.67857377, -0.59100499],
       ...,
       [ 0.32325767, -2.28981908,  0.40853234],
       [-0.17069834, -2.06444629,  0.41985608],
       [-0.27295183,  0.82636104, -0.25513213]])

In [153]:
# Create a DataFrame with the three principal components.
pcs_df = pd.DataFrame(crypto_pca)
pcs_df['id_num'] = crypto_id
pcs_df = pcs_df.set_index('id_num')
pcs_df.index.name = None
pcs_df.rename(columns = {0: "PC 1",1:"PC 2",2:"PC 3"},inplace=True)
pcs_df.head(10)

,PC 1,PC 2,PC 3
42,-0.333626,1.017520,-0.601464
404,-0.316968,1.017689,-0.601799
1337,2.292287,1.678574,-0.591005
BTC,-0.150938,-1.321564,0.162901
ETH,-0.153628,-2.070344,0.362245
LTC,-0.156112,-1.136316,-0.049173
DASH,-0.412801,1.231005,-0.478605
XMR,-0.148146,-2.205022,0.471019
ETC,-0.152070,-2.070440,0.362243
ZEC,-0.170697,-2.064446,0.419856


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [154]:
# Create an elbow curve to find the best value for K.
#Find the best value for K
inertia = []
k = list(range(1,11))

#calculate the inertia for the range of K values
for i in k:
    km=KMeans(n_clusters = i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)
    
#Create the elbow curve
elbow_data = {"k":k,"inertia":inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k",y="inertia",xticks=k,title="Elbow Curve")

C:\Users\mgsri\anaconda3\envs\mlenv\lib\site-packages\sklearn\cluster\_kmeans.py:1037: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.



:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [155]:
# Initialize the K-Means model.
model = KMeans(n_clusters = 4, random_state = 0)

# Fit the model
model.fit(pcs_df)

# Predict clusters
predictions = model.predict(pcs_df)
predictions
#fit the model

array([1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1,
       1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0,
       1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1,
       0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1,
       1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1,
       1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1,

In [156]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = pd.concat([crypto_df, pcs_df],axis=1)
clustered_df
#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df["CoinName"]=names_df["CoinName"]
#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df["Class"] = model.labels_
clustered_df.rename(columns = {0: "PC 1",1:"PC 2",2:"PC 3"},inplace=True)

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)


(532, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,Class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.333626,1.017520,-0.601464,42 Coin,1
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.316968,1.017689,-0.601799,404Coin,1
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.292287,1.678574,-0.591005,EliteCoin,1
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.150938,-1.321564,0.162901,Bitcoin,0
ETH,Ethash,PoW,1.076842e+08,0,-0.153628,-2.070344,0.362245,Ethereum,0
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.156112,-1.136316,-0.049173,Litecoin,0
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.412801,1.231005,-0.478605,Dash,1
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.148146,-2.205022,0.471019,Monero,0
ETC,Ethash,PoW,1.133597e+08,210000000,-0.152070,-2.070440,0.362243,Ethereum Classic,0
ZEC,Equihash,PoW,7.383056e+06,21000000,-0.170697,-2.064446,0.419856,ZCash,0


In [157]:
clustered_df['Class'].value_counts()

1    287
0    238
3      6
2      1
Name: Class, dtype: int64

In [158]:
clustered_df.loc[clustered_df["Algorithm"].isnull()==True].count()
clustered_df.columns.tolist()

['Algorithm',
 'ProofType',
 'TotalCoinsMined',
 'TotalCoinSupply',
 'PC 1',
 'PC 2',
 'PC 3',
 'CoinName',
 'Class']

### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [159]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    clustered_df,
    x='PC 1',
    y='PC 2',
    z='PC 3',
    color = "Class",
    symbol = "Class",
    width =800,
    hover_name="CoinName",
    hover_data={"Algorithm"},
    
)
fig.update_layout(legend=dict(x=0,y=1))
fig.show()



In [160]:
# Create a table with tradable cryptocurrencies.
clustered_df.hvplot.table(columns=["CoinName","Algorithm","ProofType","TotalCoinSupply","TotalCoinsMined","Class"],sortable=True, selectable=True)


:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]

In [161]:
# Print the total number of tradable cryptocurrencies.
tradable_count = clustered_df["CoinName"].count()
print(f"There are {tradable_count} tradable cryptocurrencies.")

There are 532 tradable cryptocurrencies.


In [162]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
clustered_scaled = clustered_df.copy()
two_cols_to_scale = clustered_df[["TotalCoinSupply","TotalCoinsMined"]]

two_columns_scaled = MinMaxScaler().fit_transform(two_cols_to_scale)
two_columns_scaled


array([[4.20000000e-11, 0.00000000e+00],
       [5.32000000e-04, 1.06585544e-03],
       [3.14159265e-01, 2.95755135e-02],
       ...,
       [1.40022261e-03, 9.90135079e-04],
       [2.10000000e-05, 7.37028150e-06],
       [1.00000000e-06, 1.29582282e-07]])

In [163]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
two_columns_scaled = pd.DataFrame(two_columns_scaled,columns=["TotalCoinSupply_scaled","TotalCoinsMined_scaled"])

two_columns_scaled['id_num'] = crypto_id
two_columns_scaled = two_columns_scaled.set_index('id_num')
two_columns_scaled.index.name = None

# Add predicted values onto the original dataframe
# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
clustered_scaled = pd.concat([clustered_df, two_columns_scaled],axis=1)
#Drop extra columns
clustered_scaled.drop(["TotalCoinsMined","TotalCoinSupply","Algorithm", "ProofType","PC 1", "PC 2", "PC 3"],axis=1,inplace=True)
#Rename scaled columns
clustered_scaled.rename(columns = {"TotalCoinsMined_scaled": "TotalCoinsMined","TotalCoinSupply_scaled":"TotalCoinSupply"},inplace=True)
#Reorder columns in dataframe
neworder = ["TotalCoinSupply","TotalCoinsMined","CoinName","Class"]
clustered_scaled = clustered_scaled.reindex(columns=neworder)
#Show dataframe
clustered_scaled.head(10)





,TotalCoinSupply,TotalCoinsMined,CoinName,Class
42,4.200000e-11,0.000000,42 Coin,1
404,5.320000e-04,0.001066,404Coin,1
1337,3.141593e-01,0.029576,EliteCoin,1
BTC,2.100000e-05,0.000018,Bitcoin,0
ETH,0.000000e+00,0.000109,Ethereum,0
LTC,8.400000e-05,0.000064,Litecoin,0
DASH,2.200000e-05,0.000009,Dash,1
XMR,0.000000e+00,0.000017,Monero,0
ETC,2.100000e-04,0.000115,Ethereum Classic,0
ZEC,2.100000e-05,0.000007,ZCash,0


In [164]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
clustered_scaled.hvplot.scatter(x="TotalCoinsMined",y="TotalCoinSupply",by="Class",hover_cols = ["CoinName"])

:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)